In [3]:
# YOLO object detection
import cv2 as cv
import numpy as np
import os
import pandas as pd

In [5]:
# get random colors for boxes
np.random.seed(42)
colors = np.random.randint(0, 255, size=(10, 3), dtype='uint8')

In [3]:
# Give the configuration and weight files for the model and load the network.
configPath = '../Data/YOLO/num_reader/SVHN3_custom-yolov4-tiny-detector.cfg'
weightsPath = '../Data/Yolo/num_reader/SVHN3_custom-yolov4-tiny-detector_best.weights'

net = cv.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)

# determine the output layer
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [15]:
def create_labeled_image(image, input_path, out_path):    
    # read in image and construct a blob from the image
    img = cv.imread(input_path + image)
    blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

    # get detections
    net.setInput(blob)
    outputs = net.forward(ln)

    # initialize lists
    boxes = []
    confidences = []
    classIDs = []
    
    # initialize image dimensions
    h_img, w_img = img.shape[:2]

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Only keep detection if it is for a person with high confidence
            if confidence > 0.5:
                box = detection[:4] * np.array([w_img, h_img, w_img, h_img])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                box = [x, y, int(width), int(height)]
                boxes.append(box)
                confidences.append(float(confidence))
                classIDs.append(classID)
                
    # get indices of final bounding boxes  
    indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    # initialize list for digit position and value
    bib_digit_loc = []
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            
            cv.rectangle(img, (x, y), (x + w, y + h), color, 1)
            text = "{}: {:.4f}".format(classIDs[i], confidences[i])
            cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            bib_digit_loc.append((x, str(classIDs[i])))
        
        # save annotated image
        cv.imwrite(out_path+image[:-4]+'_'+'detected'+'.JPG', img)
        
        # write race bib number to file
        bib_digit_loc.sort()
        rbn_pred = int(''.join([i[1] for i in bib_digit_loc]))
        #orig_image = '_'.join(image.split('_')[:2]) + '.JPG'
        rbn_pred_file = open(out_path + 'rbn_preds.txt', 'a')
        rbn_pred_file.writelines(f"{image},{rbn_pred}\n")

In [16]:
images_path = '../Data/Bibs/'
images = [file for file in os.listdir(images_path) if file[-3:]=='JPG']

output_path = '../Data/Nums/'

In [17]:
#check for existing bib_numbers.txt and remove if exists
if os.path.exists(output_path + 'rbn_preds.txt'):
    os.remove(output_path + 'rbn_preds.txt')

In [18]:
for image in images:
    create_labeled_image(image, images_path, output_path)

# Validation
---

## Training Validation
![Digit Detection](../Data/SVHN_training_validation.png "Digit Detection")

In [4]:
true_df = pd.read_csv('../Data/Bibs/bib_numbers.txt', delimiter=',', 
                      index_col=0, names=['image', 'rbn'])
true_df.head()

,rbn
image,
set1_62_bib_1.JPG,941
set1_76_bib_1.JPG,3621
set1_89_bib_1.JPG,1703
set1_88_bib_1.JPG,1442
set1_77_bib_1.JPG,847


In [20]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, set1_62_bib_1.JPG to set3_42_bib_1.JPG
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   rbn     290 non-null    int64
dtypes: int64(1)
memory usage: 4.5+ KB


In [5]:
pred_df = pd.read_csv('../Data/Nums/rbn_preds.txt', delimiter=',', 
                      index_col=0, names=['image', 'pred_rbn'])
pred_df.head()

,pred_rbn
image,
set3_21_bib_2.JPG,3054
set1_29_bib_1.JPG,130
set2_27_bib_2.JPG,20927
set2_50_bib_1.JPG,89
set3_56_bib_1.JPG,2244


In [22]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286 entries, set3_21_bib_2.JPG to set3_03_bib_6.JPG
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   pred_rbn  286 non-null    int64
dtypes: int64(1)
memory usage: 4.5+ KB


In [6]:
all_df = pd.merge(true_df, pred_df, on='image', how='left')

In [24]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, set1_62_bib_1.JPG to set3_42_bib_1.JPG
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rbn       290 non-null    int64  
 1   pred_rbn  286 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.8+ KB


#### Accurate Predictions

In [25]:
all_df.loc[all_df['rbn'] == all_df['pred_rbn']]

,rbn,pred_rbn
image,,
set1_76_bib_1.JPG,3621,3621.0
set1_75_bib_1.JPG,1676,1676.0
set1_61_bib_1.JPG,1679,1679.0
set1_48_bib_1.JPG,663,663.0
set1_60_bib_1.JPG,1404,1404.0
...,...,...
set3_55_bib_2.JPG,4624,4624.0
set3_43_bib_1.JPG,2074,2074.0
set3_57_bib_2.JPG,4183,4183.0


#### Inaccurate Predictions

In [26]:
all_df.loc[all_df['rbn'] != all_df['pred_rbn']]

,rbn,pred_rbn
image,,
set1_62_bib_1.JPG,941,4.0
set1_89_bib_1.JPG,1703,7703.0
set1_88_bib_1.JPG,1442,442.0
set1_77_bib_1.JPG,847,84.0
set1_63_bib_1.JPG,941,94.0
...,...,...
set3_40_bib_1.JPG,3271,327.0
set3_40_bib_2.JPG,331,33.0
set3_55_bib_1.JPG,2331,233.0


#### No Prediction

In [30]:
all_df.loc[all_df['pred_rbn'].isna()]

,rbn,pred_rbn
image,,
set1_28_bib_1.JPG,311,NaN
set1_16_bib_1.JPG,1463,NaN
set1_17_bib_1.JPG,1463,NaN
set1_07_bib_1.JPG,979,NaN


#### Accuracy

In [31]:
196/290

0.6758620689655173